In [4]:
import torch

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

Running on cpu


## **Reading CSV file**

In [6]:
import csv
import re

def extract_first_name(csv_file):
    unique=set()
    names = []
    with open(csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if len(row['Name'].split())>0:
              full_name = row['Name'].split()
              for name in full_name:
                name=name.lower()
                if re.match("^[a-zA-Z]+$", name) and name not in unique and len(name)>3:

                    names.append(name)
                    unique.add(name)
    return names

csv_file = '/kaggle/input/data-name/Gender_Data.csv'
names = extract_first_name(csv_file)
names[:10]

['aaban',
 'aabharan',
 'aabhas',
 'aabhat',
 'aabheer',
 'abheer',
 'aabher',
 'aabi',
 'aabilesh',
 'aabir']

In [7]:
maxx=0
maxx_name=''
for i in names:
  if len(i)>maxx:
    maxx=len(i)
    maxx_name=i

print(maxx)
print(maxx_name)

24
parthadhwajagrasamvasine


In [8]:
names.remove(maxx_name)

In [9]:
maxx=0
maxx_name=''
for i in names:
  if len(i)>maxx:
    maxx=len(i)
    maxx_name=i

print(maxx)
print(maxx_name)

22
dooshanatrishirohantre


In [10]:
print(len(names))

53750


In [11]:
import random 
random.shuffle(names)
words = names[:41000]

In [12]:
maxx_name_len=0
maxx_name=''
for i in words:
  if len(i)>maxx_name_len:
    maxx_name_len=len(i)
    maxx_name=i

print(maxx_name_len)
print(maxx_name)

22
nishumbhashumbhahanani


---------------------------------------------------------------------------------------

In [13]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
import pickle

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

# Save the dictionary to a pickle file
with open('/kaggle/working/conversion.pkl', 'wb') as f:
    pickle.dump((itos, vocab_size), f)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [15]:
import random
random.seed(42)
random.shuffle(words)

In [16]:
block_size = 16

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  X = torch.tensor(X,device=device)
  Y = torch.tensor(Y,device=device)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     
Xdev, Ydev = build_dataset(words[n1:n2])   
Xte,  Yte  = build_dataset(words[n1:])     

torch.Size([297184, 16]) torch.Size([297184])
torch.Size([37179, 16]) torch.Size([37179])
torch.Size([74256, 16]) torch.Size([74256])


In [18]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

................ --> j
...............j --> i
..............ji --> a
.............jia --> s
............jias --> h
...........jiash --> i
..........jiashi --> n
.........jiashin --> i
........jiashini --> .
................ --> k
...............k --> a
..............ka --> r
.............kar --> k
............kark --> a
...........karka --> .
................ --> k
...............k --> e
..............ke --> e
.............kee --> r
............keer --> a


In [19]:
import torch

# -----------------------------------------------------------------------------------------------
class Linear:

  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), device=device) / fan_in**0.5 # note: kaiming init
    self.bias = torch.zeros(fan_out, device=device) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# -----------------------------------------------------------------------------------------------
class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim, device=device)
    self.beta = torch.zeros(dim, device=device)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim, device=device)
    self.running_var = torch.ones(dim, device=device)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

# -----------------------------------------------------------------------------------------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Embedding:

  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim), device=device)

  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out

  def parameters(self):
    return [self.weight]

# -----------------------------------------------------------------------------------------------
class FlattenConsecutive:

  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T//self.n, C*self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out

  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Sequential:

  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out

  def parameters(self):
    # get parameters of all layers and stretch them out into one list
    return [p for layer in self.layers for p in layer.parameters()]

In [20]:
torch.manual_seed(42)

In [21]:
n_embd = 32
n_hidden = 128

model = Sequential([
  Embedding(vocab_size, n_embd),
  FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=True), torch.nn.Dropout(p=0.2),BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=True), torch.nn.Dropout(p=0.2),BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=True), torch.nn.Dropout(p=0.1),BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=True), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, vocab_size),
])

with torch.no_grad():
  model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))
for p in parameters:
  p.requires_grad = True

112379


In [29]:
max_steps = 100000
batch_size = 32
lossi = []
lossi_val = []
min_loss=10

for i in range(max_steps):
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  print(f"ix:{ix}")
  Xb, Yb = Xtr[ix], Ytr[ix]
  Yb = Ytr[ix]
  print(f"Yb:{Yb}")
  logits = model(Xb)
  print(f"logits:{logits}")
  loss = F.cross_entropy(logits, Yb)
  print(f"Loss:{loss}")
  break
  for p in parameters:
    p.grad = None

  loss.backward()

  lr=0.1
  for p in parameters:
    p.data += -lr * p.grad

  if i % 2000 == 0:
    logits_val = model(Xdev)
    loss_val = F.cross_entropy(logits_val, Ydev)
    lossi_val.append(loss_val.log10().item())
    logits_train = model(Xtr)
    loss_train = F.cross_entropy(logits_train, Ytr)

    print(f'Validation loss : {loss_val.item():.4f} ; Training loss : {loss_train.item():.4f} ')

    if loss_val.item()<min_loss:
      torch.save(model, '/kaggle/working/model1.pth')
      min_loss=loss_val.item()
  lossi.append(loss.log10().item())

print("Minimum loss obtained on training set : ",min_loss," corresponding model has been saved at /kaggle/working/model1.pth")

ix:tensor([ 46674,  78449, 210775,  95132, 104172, 171525, 284540,  12643,  87028,
        139145,  17358,   9558, 196395, 206779,  20051, 160235, 195005, 152890,
        180038, 243461,   1700, 102866,  88109, 171040, 108279, 280938, 171837,
         87190,  63550,  84245, 101667, 287284])
Yb:tensor([14, 14,  1, 20,  9, 16, 13,  9,  0,  0,  1, 23,  1,  8, 10,  1,  1, 25,
        19, 10,  0, 16, 21,  1, 11, 22, 10,  1, 26,  9, 10,  9])
logits:tensor([[ 9.0395e-03,  6.5382e-02, -3.8657e-02, -2.6899e-02, -8.1669e-02,
         -3.7969e-02,  1.4144e-01, -1.2350e-01, -9.4955e-02, -4.6527e-02,
         -7.5448e-03,  5.8065e-02, -2.3412e-02,  8.2098e-03,  8.3538e-02,
          7.2446e-03, -3.0023e-03, -2.2411e-03, -8.9048e-03, -3.2845e-02,
         -4.8883e-02, -2.7109e-02,  1.4985e-02, -4.7366e-02, -1.6246e-02,
         -1.4239e-02,  1.1965e-01],
        [-1.8692e-02, -1.1129e-01, -4.0213e-02, -2.7340e-02,  2.4323e-02,
         -9.2991e-02, -4.7280e-02,  9.6571e-02, -1.4046e-02,  1.3456e-02,

In [82]:
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

# split_loss('train')
# split_loss('val')
split_loss('test')

test 1.8645883798599243


In [83]:
torch.save(model,'/kaggle/working/model3.pth')

In [84]:
best_model = torch.load('/kaggle/working/model3.pth')

In [85]:
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = best_model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('test')

test 1.8645048141479492


In [86]:
for layer in best_model.layers:
  layer.training = False

In [89]:
predicted=[]
while len(predicted)<100:

    out = []
    context = [0] * block_size
    while True:
      logits = best_model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    name=''.join(itos[out[i]] for i in range(len(out)-1))
    if name not in words:
      predicted.append(name)


In [90]:
for i in predicted:
  print(i)

thurmitha
maavyukan
ambeeth
mohthrakash
prathivethi
vathuga
layazhini
velamma
manveen
maharmshi
harvika
dumbiraj
paryamila
vibaryani
vaisderesh
maleeesh
athiramala
aarvala
aasharat
yathumiya
pheshka
madham
tamilya
dhamutaraj
aruveren
jinnakumar
kritha
chalajan
jasali
chandravathy
manjali
sharakshan
saanghi
mishaika
ajavardha
aveeravaran
mohilaa
zandika
chembal
aridhri
pahaluxuthra
aathang
shanalaryan
rinuesanth
thaarshini
ahatsan
sarbhanu
akshisha
jalakshmi
nivajaya
geeiyaanan
nithur
charakshi
thuvashriyan
ramahi
indhuri
nayavati
aruneetha
devimara
jayravarshyan
patman
nivamanha
khinkodhan
vijaytha
ganyaath
seemali
gulesh
navandi
thaar
vayathalini
shuvam
maashmiyash
jaysuth
prabhudeeban
uddatrishna
anumritt
samaity
abhithika
viurashiharee
devakaranaventhan
amilaavanak
vivarshayan
vashal
vaishavarshan
aani
kaliyasani
jairana
nirishavan
ashner
maheemanan
illakumar
uviva
kaavyamazhi
amandree
rajambari
adheevan
kayalkarasi
anursha
aatheran
aashankumar
